# 👩🏻‍🔬 Offline inference pipeline: Computing item embeddings

In this notebook you will compute the candidate embeddings and populate a Hopsworks feature group with a vector index.

In [1]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.filterwarnings("ignore")

from loguru import logger
from recsys.config import settings
from recsys.gcp.vertex_ai import model_registry
from recsys.gcp.bigquery import client as bq_client
from recsys.gcp.feature_store import client as fs_client
from recsys.core.embeddings.computation import compute_embeddings
from recsys.gcp.feature_store.datasets import create_training_dataset
from recsys.core.embeddings.preprocessing import preprocess_candidates
from recsys.data.preprocessing.splitting import train_validation_test_split

## ☁️ Connect to Vertex AI Feature Online Store

In [2]:
fos = fs_client.get_client()

2025-02-20 11:54:53.675 | INFO     | recsys.gcp.feature_store.client:get_client:31 - Retrieving Feature Store from us-central1/recsys-dev-gonzo-2/recsys_feature_store_dev


In [3]:
trans_fv, articles_fv, customers_fv, _ = fs_client.get_feature_views(fos)

# Computing candidate embeddings

You start by computing candidate embeddings for all items in the training data.

First, you load your candidate model. Recall that you uploaded it to the Vertex AI Model Registry in previous steps:

In [4]:
candidate_model, candidate_features = model_registry.get_model(
    model_name="candidate_tower_v1"
)

2025-02-20 11:54:59.269 | INFO     | recsys.gcp.vertex_ai.model_registry:get_model:164 - Downloading '3333276152230838272' version gs://recsys-dev-gonzo-2-vertex-staging-us-central1/vertex_ai_auto_staging/2025-02-19-14:37:18.875
2025-02-20 11:55:06.715 | INFO     | recsys.gcp.vertex_ai.model_registry:get_model:175 - Extracted 3 input features from model


### Get candidates data

Now, we get the training retrieval data containing all the features required for the candidate embedding model.

In [5]:
training_data = create_training_dataset(trans_fv, articles_fv, customers_fv)

2025-02-20 11:55:06.770 | INFO     | recsys.gcp.feature_store.datasets:create_training_dataset:46 - Fetching transactions data...
2025-02-20 11:55:06.771 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:185 - Fetching data from feature view: transactions
2025-02-20 11:55:07.128 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:198 - Executing query: SELECT customer_id, article_id, t_dat, price, month_sin, month_cos FROM `recsys-dev-gonzo-2.recsys_dataset.recsys_transactions`
2025-02-20 11:55:11.324 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:201 - DataFrame shape: (23799, 6)
2025-02-20 11:55:11.325 | INFO     | recsys.gcp.feature_store.datasets:create_training_dataset:51 - Fetching customer data...
2025-02-20 11:55:11.325 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:185 - Fetching data from feature view: customers
2025-02-20 11:55:11.587 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:198 - Executing qu

In [6]:
train_df, val_df, test_def, _, _, _ = train_validation_test_split(
    df=training_data,
    validation_size=settings.TWO_TOWER_DATASET_VALIDATION_SPLIT_SIZE,
    test_size=settings.TWO_TOWER_DATASET_TEST_SPLIT_SIZE,
)

2025-02-20 11:55:20.732 | INFO     | recsys.data.preprocessing.splitting:train_validation_test_split:316 - Split complete: train=19005 rows, validation=2389 rows, test=2405 rows


In [7]:
train_df.head(3)

customer_id,article_id,t_dat,price,month_sin,month_cos,age,club_member_status,age_group,garment_group_name,index_group_name
str,str,i64,f64,f64,f64,f64,str,str,str,str
"""621788f7946826d475ae634d5138fd…","""511105002""",0,0.022017,0.5,0.866025,35.0,"""ACTIVE""","""26-35""","""Accessories""","""Ladieswear"""
"""f17d07ee3b52dc06ba23e5dbd0621a…","""619506001""",0,0.013542,0.5,0.866025,33.0,"""ACTIVE""","""26-35""","""Accessories""","""Sport"""
"""c7d488a1e7c4a6141e313199d41e55…","""701973001""",0,0.015237,0.5,0.866025,42.0,"""ACTIVE""","""36-45""","""Accessories""","""Baby/Children"""


### Compute embeddings

Next you compute the embeddings of all candidate items that were used to train the retrieval model.

In [8]:
item_df = preprocess_candidates(train_df, candidate_features)
item_df.head(3)

garment_group_name,article_id,index_group_name
str,str,str
"""Jersey Fancy""","""708536005""","""Divided"""
"""Woven/Jersey/Knitted mix Baby""","""633919004""","""Baby/Children"""
"""Knitwear""","""636938005""","""Ladieswear"""


In [9]:
embeddings_df = compute_embeddings(item_df, candidate_model)
embeddings_df.head()

article_id,embeddings
i64,list[f64]
708536005,"[-0.833923, -2.762513, … 2.363503]"
633919004,"[0.309819, -0.272627, … 0.620053]"
636938005,"[-0.357415, 0.367256, … 0.365261]"
754370005,"[-0.71657, 0.054884, … 0.572164]"
697902001,"[-2.347109, -2.577516, … 2.290054]"


# <span style="color:#ff5f27">Create Vertex AI Embedding Index </span>

Now you are ready to create a feature group for your candidate embeddings.

To begin with, you need to create your Embedding Index where you will specify the name of the embeddings feature and the embeddings length.
Then you attach this index to the FV.

In [10]:
logger.info("Uploading 'candidates' Feature to BigQuery.")
bq_client.load_features(candidates_df=embeddings_df)
logger.info("✅ Uploaded 'candidates' Feature to BigQuery!")

2025-02-20 11:55:20.988 | INFO     | __main__:<module>:1 - Uploading 'candidates' Feature to BigQuery.
2025-02-20 11:55:20.993 | DEBUG    | recsys.gcp.bigquery.client:convert_types:60 - Converted article_id to STRING
2025-02-20 11:55:20.993 | DEBUG    | recsys.gcp.bigquery.client:convert_types:60 - Converted embeddings to FLOAT64
2025-02-20 11:55:21.002 | INFO     | recsys.core.embeddings.storage:process_for_storage:33 - Processed embeddings in embeddings
2025-02-20 11:55:21.003 | DEBUG    | recsys.gcp.bigquery.client:upload_dataframe:93 - DataFrame types before upload:
2025-02-20 11:55:21.003 | DEBUG    | recsys.gcp.bigquery.client:upload_dataframe:95 - article_id: object
2025-02-20 11:55:21.003 | DEBUG    | recsys.gcp.bigquery.client:upload_dataframe:95 - embeddings: object
2025-02-20 11:55:27.511 | INFO     | recsys.gcp.bigquery.client:upload_dataframe:109 - Loaded 12024 rows to recsys-dev-gonzo-2.recsys_dataset.recsys_candidates
2025-02-20 11:55:27.513 | INFO     | recsys.gcp.bigqu